In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
import random,itertools
import fastjet

delphesDir = os.path.abspath("../../MG5/Delphes")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT
import xml.etree.ElementTree as ET


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

Welcome to JupyROOT 6.28/06


True

In [2]:
inputFile = list(glob.glob('../../processFolders/sm_ttmatch_lo/Events/run_01/*_delphes_events.root'))[0]
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
f.Close()

In [6]:
etamax=2.0
pTmin=355.0

atlas_bins = [355.0,381.0,420.0,478.0,549.0,633.0,720.0,836.0,2000.0]
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
fatjetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)

f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
ntotal = 0
totalweightPB = 0.0
pTT = []
weights = []
for ievt in range(nevts):    
    error = False
    ntotal += 1
    tree.GetEntry(ievt)
    weightPB = tree.Weight.At(1).Weight
    weightPB = weightPB
    totalweightPB += weightPB

    genParticles = tree.Particle
    # Filter tops:
    tops = {}
    # topList = {}
    # Filter Ws
    # wList = {}
    for iptc in range(genParticles.GetEntries()):
        ptc = genParticles.At(iptc)
        if abs(ptc.PID) == 6:
            # topList[iptc] = ptc # Store all tops generated during event evolution
            tops[ptc.PID] = ptc # Store only the last top/anti-top
        # elif abs(ptc.PID) == 24:
            # wList[iptc] = ptc

    # Get top and W decays:
    topDecays = {}
    for itop,top in tops.items():
        topDecays[top.PID] = []
        for idaugther in range(top.D1,top.D2+1):
            ptc = genParticles.At(idaugther)
            if abs(ptc.PID) == 5:
                topDecays[top.PID].append(ptc)
            elif abs(ptc.PID) == 24:
                w = ptc
                d1 = w.D1
                d2 = w.D2+1
                daughter1 = genParticles.At(d1)
                while daughter1.PID == w.PID:
                    d1 = daughter1.D1
                    d2 = daughter1.D2+1
                    daughter1 = genParticles.At(d1)
                for idaugther2 in range(d1,d2):
                    ptcD = genParticles.At(idaugther2)
                    topDecays[top.PID].append(ptcD)
            elif abs(ptc.PID) == 6:
                print('Error getting top decays')
                error = True
                break
    if error:
        break
    
    # Select events with one lepton and one hadronic top:
    topH = None
    topLep = None
    for topPID,daughters in topDecays.items():
            dPIDs =  [abs(ptc.PID) for ptc in daughters]
            if not 5 in dPIDs:
                continue # Skip rare decays to W+c
            if (11 in dPIDs) or (13 in dPIDs):
                topLep = topPID
            elif max(dPIDs) <= 5:
                topH = topPID

    
    if topH is None or topLep is None:
        continue
   

    # Hadronic top:    
    # Regular jets:
    quarks = [ptc for ptc in topDecays[topH]]
    jetArray = [fastjet.PseudoJet(q.Px,q.Py,q.Pz,q.E) for q in quarks if abs(q.Eta) < 4.5]
    for ij,j in enumerate(jetArray):
        j.set_user_index(quarks[ij].PID)
    cluster = fastjet.ClusterSequence(jetArray, jetdef)
    jets = cluster.inclusive_jets(ptmin = 26.0)
    jets = [j for j in jets if abs(j.eta()) < 2.5]
    if len(jets) == 0:
        continue

        
    # ## Fat jet:    
    jetArray = [fastjet.PseudoJet(j.px(),j.py(),j.pz(),j.E()) for j in jets]
    for ij,j in enumerate(jetArray):
        for q in jets[ij].constituents():
            if abs(q.user_index()) == 5:
                j.set_user_index = 5 # Tag the regular jets containing a b-quark
    clusterFat = fastjet.ClusterSequence(jetArray, fatjetdef)
    if len(clusterFat.inclusive_jets()) == 0:
        continue
    fatJet = sorted([j for j in clusterFat.inclusive_jets()], key = lambda j: j.pt(), reverse=True)[0]
    if not (120. < fatJet.m() < 220.):
        continue
    if fatJet.pt() < pTmin:
        continue
    if abs(fatJet.eta()) > etamax:
        continue
    hasB = False
    for q in fatJet.constituents():
        if q.user_index() == 5:
            hasB = True
    if not hasB:
        continue

    
    
    # Leptonic top:
    leptons = [ptc for ptc in topDecays[topLep] if abs(ptc.PID) in [11,13]]
    if len(leptons) != 1:
        error = True
        print('Error getting leptons')
        break
    if error:
        break
    lepton = leptons[0]
    dRlep = min([np.sqrt((lepton.Eta-j.eta())**2 + (lepton.Phi-j.phi())**2) for j in jets])
    # Skip events where lepton overlaps to jet
    if dRlep < 0.4:
        continue

    pTlepton = lepton.PT
    etaLep = np.abs(lepton.Eta)
    if pTlepton < 27.0:
        continue
    if etaLep > 2.5:
        continue

    pTT.append(tops[topH].PT)
    weights.append(weightPB)
    
weights = np.array(weights)
pTtHist,_ = np.histogram(pTT,weights=weights,bins=atlas_bins)
pTtHistError,_ = np.histogram(pTT,weights=weights**2,bins=atlas_bins)
pTtHistError = np.sqrt(pTtHistError)

data = np.array(list(zip(atlas_bins[:-1],atlas_bins[1:],pTtHist,pTtHistError)))
    

In [7]:
kfac = 832./totalweightPB
print('xsec (pb) = %1.3e' %totalweightPB)
print('fiducial xsec (pb) = %1.3e' %sum(weights))
print('fiducial xsec*kfac (pb) = %1.3e' %(sum(weights)*kfac))


xsec (pb) = 4.653e+02
fiducial xsec (pb) = 0.000e+00
fiducial xsec*kfac (pb) = 0.000e+00
